# 📓 01 - Modelos baseados em prompt (zero-shot e few-shot) vs. Fine-tuning

Este notebook demonstra, na prática, como usar modelos baseados em prompts (zero-shot e few-shot) para tarefas de NLP, comparando-os ao conceito de fine-tuning (teórico, pois exige dataset e GPU).

---

## ⚙️ Setup inicial

Vamos instalar e importar as bibliotecas necessárias.

```python
# Instalar a biblioteca se necessário
pip install transformers

from transformers import pipeline
```

In [1]:
from transformers import pipeline

c:\Users\vinicius\Documents\Projetos\RAG-study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🔎 1️⃣ Exemplo zero-shot: Summarization

In [2]:
# Carregar um pipeline de summarization com modelo pré-treinado
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Texto para resumir
texto = """
Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.
Ele busca dados em fontes externas para fornecer respostas mais relevantes e atualizadas.
Essa técnica é muito útil em aplicações que precisam de dados dinâmicos e confiáveis.
"""

# Gerar resumo
resumo = summarizer(texto, max_length=50, min_length=25, do_sample=False)
print("🔹 Resumo gerado:", resumo[0]['summary_text'])

Device set to use cpu


🔹 Resumo gerado: Retrieval-Augmented Generation (RAG) combina recuperação de informações and geração of linguagem natural.Ele busca dados em fontes externas para


## ✏️ 2️⃣ Few-shot prompting manual (usando prompt formatado)
Neste exemplo, vamos "simular" um few-shot prompting manual, usando apenas texto, para mostrar como dar exemplos ajuda o modelo a entender melhor.

In [6]:
# Prompt few-shot: Incluímos exemplos no prompt!
few_shot_prompt = """
Você é um assistente que resume textos em 1 frase.

Exemplo 1:
Texto: O sol nasce no leste e se põe no oeste. Ele aquece a Terra.
Resumo: O sol aquece a Terra, surgindo no leste e se pondo no oeste.

Agora, Texto:
Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.
Ele busca dados em fontes externas para fornecer respostas mais relevantes e atualizadas.
Essa técnica é muito útil em aplicações que precisam de dados dinâmicos e confiáveis.
"""

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [7]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Você é um assistente útil."},
        {"role": "user", "content": few_shot_prompt}
    ]
)

print("🔹 Resumo gerado:", response.choices[0].message.content.strip())

🔹 Resumo gerado: RAG combina recuperação de informações e geração de linguagem natural para fornecer respostas relevantes e atualizadas, sendo útil em aplicações que necessitam de dados dinâmicos.


## 🛠️ 3️⃣ (Teórico) Fine-tuning com HuggingFace
Fine-tuning exige dados rotulados e geralmente GPUs. Vamos mostrar apenas o pipeline conceitual:

```
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=meu_dataset_treino,
    eval_dataset=meu_dataset_val
)

# Iniciar treinamento
trainer.train()
```